# Gerenciar Experimentos — v1, v2, v3

Painel prático para:
- Excluir modelos/mapeamentos gerados (todas as versões)
- Retreinar v1, v2 (executando os notebooks) e v3 (script)
- Gerar textos comparativos nas 3 versões e salvar relatório em `comparisons/`

Observação: v1 e v2 rodam via execução dos notebooks; v3 roda via script.


In [ ]:
# SETUP
import os, sys, shutil, subprocess, json, time
from pathlib import Path
from datetime import datetime

REPO = Path.cwd()
V1 = REPO / 'versions' / 'v1-char-rnn'
V2 = REPO / 'versions' / 'v2-char-lm'
V3 = REPO / 'versions' / 'v3-brwac'

NB_V1 = V1 / 'notebooks' / 'llm.ipynb'
NB_V2 = V2 / 'notebooks' / 'llm_v2.ipynb'
SCRIPT_V3 = V3 / 'scripts' / 'treinar_com_brwac.py'

MODELS_DIRS = [V1/'models', V2/'models', V3/'models']
MAPS_DIRS = [V1/'mappings', V2/'mappings', V3/'mappings']
COMP_DIR = REPO / 'comparisons'
COMP_DIR.mkdir(exist_ok=True)

print('Repo:', REPO)
print('v1:', V1)
print('v2:', V2)
print('v3:', V3)


In [ ]:
# UTIL: limpar artefatos (.keras/.pkl)
def clean_artifacts():
    removed = []
    for d in MODELS_DIRS + MAPS_DIRS:
        d.mkdir(parents=True, exist_ok=True)
        for p in d.glob('*'):
            if p.is_file():
                try:
                    p.unlink()
                    removed.append(str(p))
                except OSError:
                    pass
    return removed

def _run(cmd: list, cwd: Path | None = None):
    print('[RUN]', ' '.join(map(str, cmd)))
    res = subprocess.run(cmd, cwd=cwd or REPO, text=True, capture_output=True)
    print(res.stdout)
    if res.returncode != 0:
        print(res.stderr)
        raise RuntimeError(f'Command failed: {cmd}')
    return res


In [ ]:
# RETREINO
def retrain_v1():
    # Executa o notebook v1 inplace
    cmd = [sys.executable, '-m', 'jupyter', 'nbconvert', '--to', 'notebook', '--inplace', '--execute', str(NB_V1)]
    return _run(cmd)

def retrain_v2():
    cmd = [sys.executable, '-m', 'jupyter', 'nbconvert', '--to', 'notebook', '--inplace', '--execute', str(NB_V2)]
    return _run(cmd)

def retrain_v3(epocas=10, max_textos=10000, tamanho_sequencia=160, tamanho_lstm=256, batch_size=256, validacao_split=0.05):
    cmd = [sys.executable, str(SCRIPT_V3),
           '--epocas', str(epocas),
           '--max_textos', str(max_textos),
           '--tamanho_sequencia', str(tamanho_sequencia),
           '--tamanho_lstm', str(tamanho_lstm),
           '--batch_size', str(batch_size),
           '--validacao_split', str(validacao_split)]
    return _run(cmd)


In [ ]:
# COMPARACAO
from compare_generate import run_compare, ModelSpec

def _default_specs():
    return [
        ModelSpec('v1-char-rnn', 'versions/v1-char-rnn/models/modelo_char_rnn.keras', 'versions/v1-char-rnn/mappings/mapeamentos.pkl'),
        ModelSpec('v2-char-lm', 'versions/v2-char-lm/models/modelo_char_lm_v2.keras', 'versions/v2-char-lm/mappings/mapeamentos_v2.pkl'),
        ModelSpec('v3-brwac', 'versions/v3-brwac/models/modelo_brwac.keras', 'versions/v3-brwac/mappings/mapeamentos_brwac.pkl'),
    ]

def _filter_specs(specs, only):
    if not only: return specs
    keys = [k.strip().lower() for k in only.split(',') if k.strip()]
    keep = []
    for s in specs:
        tag = 'v1' if 'v1-char-rnn' in s.name else ('v2' if 'v2-char-lm' in s.name else ('v3' if 'v3-brwac' in s.name else s.name))
        if tag.lower() in keys: keep.append(s)
    return keep

def compare_and_save(prompts, length=400, temperature=0.8, only=''):
    ts = datetime.now().strftime('%Y%m%d-%H%M%S')
    out = []
    specs = _filter_specs(_default_specs(), only)
    for p in prompts:
        out.append(f'PROMPT: {p}')
        import io, contextlib
        buf = io.StringIO()
        with contextlib.redirect_stdout(buf):
            run_compare(p, length, temperature, specs, seed=42)
        out.append(buf.getvalue())
        out.append('-'*80)
    path = COMP_DIR / f'comparacao-{ts}.txt'
    path.write_text('
'.join(out), encoding='utf-8')
    print('Salvo em:', path)
    return path


## Uso Rápido

- Para apagar artefatos: execute a célula abaixo.
- Para retreinar: execute as funções `retrain_v1()`, `retrain_v2()`, `retrain_v3(...)`.
- Para comparar: edite a lista de prompts e rode `compare_and_save(...)`.


In [ ]:
# EXCLUIR ARTEFATOS (use com cuidado)
removed = clean_artifacts()
print('Removidos:', len(removed))
for p in removed[:10]:
    print(' -', p)
if len(removed) > 10:
    print(' ...')


In [ ]:
# EXEMPLOS DE RETREINO
# retrain_v1()
# retrain_v2()
# retrain_v3(epocas=10, max_textos=10000, tamanho_sequencia=160, tamanho_lstm=512, batch_size=256, validacao_split=0.05)


In [ ]:
# COMPARACOES
prompts = [
    'O trabalho de conclusão de curso deve',
    'O dia-a-dia de um homem',
    'O trabalho da mulher',
    'Sempre vou buscar',
]
# path = compare_and_save(prompts, length=400, temperature=0.8)
# print('Relatório:', path)
